## Subquestion Query Engine as a workflow

#### Note: Create and Use environment llamindex_workflow 

In [12]:
# !pip install llama-index-core llama-index-llms-openai llama-index-embeddings-openai llama-index-readers-file 
# !pip install llama-index-utils-workflow
# llama-index-utils-workflow



In [13]:
import os, json
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex, 
    StorageContext, 
    load_index_from_storage)
from llama_index.core.tools import (
    QueryEngineTool, 
    ToolMetadata)
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Context
)
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent
from dotenv import load_dotenv
import openai
from llama_index.utils.workflow  import draw_all_possible_flows
from llama_index.llms.azure_openai import AzureOpenAI


# Define the Sub Question Query Engine as a Workflow

* Our StartEvent goes to `query()`, which takes care of several things:
  * Accepts and stores the original query
  * Stores the LLM to handle the queries
  * Stores the list of tools to enable sub-questions
  * Passes the original question to the LLM, asking it to split up the question into sub-questions
  * Fires off a `QueryEvent` for every sub-question generated

* QueryEvents go to `sub_question()`, which instantiates a new ReAct agent with the full list of tools available and lets it select which one to use.
  * This is slightly better than the actual SQQE built-in to LlamaIndex, which cannot use multiple tools
  * Each QueryEvent generates an `AnswerEvent`

* AnswerEvents go to `combine_answers()`.
  * This uses `self.collect_events()` to wait for every QueryEvent to return an answer.
  * All the answers are then combined into a final prompt for the LLM to consolidate them into a single response
  * A StopEvent is generated to return the final result

In [14]:
class QueryEvent(Event):
    question : str

class AnswerEvent(Event):
    question : str
    answer : str

class SubQuestionQueryEngine(Workflow):

    @step(pass_context=True)
    async def query(self, ctx:Context, ev:StartEvent) -> QueryEvent:
        if(hasattr(ev, "query")):
            ctx.data["original_query"] = ev.query
            print(f"Query is {ctx.data['original_query']}")

        if (hasattr(ev,"llm")):
            ctx.data["llm"] = ev.llm

        if (hasattr(ev,"tools")):
            ctx.data["tools"] = ev.tools

        response = ctx.data["llm"].complete(f"""
            Given a user question, and a list of tools, output a list of
            relevant subquestions, such that the answers to all the 
            sub-questions put together will answer the question.Respond in 
            pure JSON without any markdown, like this:
            {{
                "sub_questions": [
                    "What is the Net Profit of Infosys?",
                    "What is the Net Loss of Infosys?",
                    "What is percentage growth for Infosys?"
                ]
            }})
            Here is the User question: {ctx.data["original_query"]}

            And here is the list of tools: {ctx.data["tools"]}
            """)

        response_obj = json.loads(str(response))
        sub_questions = response_obj["sub_questions"]

        ctx.data["sub_question_count"] = len(sub_questions)

        for question in sub_questions:
            self.send_event(QueryEvent(question=question))

        return None

    @step(pass_context=True)
    async def sub_question(self,ctx:Context, ev:QueryEvent) ->AnswerEvent:
        print(f"Sub-question is {ev.question}")
        agent = ReActAgent.from_tools(tools = ctx.data["tools"],llm=ctx.data['llm'],verbose=True)
        response = agent.chat(ev.question)
        print(f"-------------{response}----------")

        return AnswerEvent(question=ev.question,answer = str(response))

    
    @step(pass_context=True)
    async def combine_answers(self,ctx:Context,ev:AnswerEvent) -> StopEvent | None:
        ready  = ctx.collect_events(ev,[AnswerEvent]*ctx.data["sub_question_count"])

        if ready is None:
            return None

        answers = "\n\n".join([f"Question : {event.question}: \n Answer: {event.answer}" for event in ready])

        prompt = f"""
            You are given an Overall question that has been split into sub-questions,
            each of which has been answered. Combine the answers to all the sub-questions

            Original question: {ctx.data['original_query']}

            Sub-questions and answers:
            {answers}
            """

        print(f"Final prompt is {prompt}")
        response = ctx.data["llm"].complete(prompt)

        print("Final response is", response)

        return StopEvent(result=str(response))



In [15]:

draw_all_possible_flows(SubQuestionQueryEngine,filename="Sub_Question_Query_Engine_workflow.html")

Sub_Question_Query_Engine_workflow.html


## Lets Display our Workflow

In [16]:
# from IPython.display import HTML
# work_flow_html = HTML(filename="Sub_Question_Query_Engine_workflow.html")
# display(work_flow_html)

from IPython.display import IFrame
IFrame(src="Sub_Question_Query_Engine_workflow.html", width=1100, height=400)

## Download data to a demo

In [17]:
### For Linux Users - 
# ! mkdir -p "./data/infy_budgets"
# !wget "https://www.infosys.com/investors/reports-filings/annual-report/form20f/documents/form20f-2022.pdf"
# !wget "https://www.infosys.com/investors/reports-filings/annual-report/form20f/documents/form20f-2022.pdf"
# !wget "https://www.infosys.com/investors/reports-filings/annual-report/form20f/documents/form20f-2022.pdf"

In [18]:
ls "data/infy_budgets/"

 Volume in drive C has no label.
 Volume Serial Number is DC49-8C99

 Directory of C:\Users\nilesh.chopda\OneDrive - Calsoft Pvt Ltd\Project\Llamaindex-Workflow\data\infy_budgets

08-08-2024  17:59    <DIR>          .
08-08-2024  17:51    <DIR>          ..
08-08-2024  17:46         1,509,673 2022-infybudget.pdf
08-08-2024  17:46         2,543,407 2023-infybudget.pdf
08-08-2024  17:45         5,359,420 2024-infybudget.pdf
               3 File(s)      9,412,500 bytes
               2 Dir(s)  76,496,584,704 bytes free


## Lets Instantiate Our Workflow

In [19]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [20]:
# from llama_index.core.readers.file.base import SimpleDirectoryReader
folder = "data/infy_budgets/"
files = os.listdir(folder)

query_engine_tools = []

for file in files:
    year = file.split("-")[0]
    index_persist_path = f"storage/budget-{year}"
    print(f"--------Document Loaded for year {year} ---------")
    if os.path.exists(index_persist_path):
        storage_context = StorageContext.from_defaults(persist_dir=index_persist_path)
        index =load_index_from_storage(storage_context)

    else:
        documents = SimpleDirectoryReader(input_files=[folder + file]).load_data()
        index = VectorStoreIndex.from_documents(documents)
        index.storage_context.persist(index_persist_path)
    print(f"--------Indexing Document for year {year} ---------")
    engine = index.as_query_engine()
    query_engine_tools.append(
        QueryEngineTool(
            query_engine=engine,
            metadata=ToolMetadata(
                name = f"budget_{year}",
                description = f"Information about Infosys' profit loss statement in {year}",
            ),
        )
    )
print(f"--------Indexing Completed! ---------")

--------Document Loaded for year 2022 ---------
--------Indexing Document for year 2022 ---------
--------Document Loaded for year 2023 ---------
--------Indexing Document for year 2023 ---------
--------Document Loaded for year 2024 ---------
--------Indexing Document for year 2024 ---------
--------Indexing Completed! ---------


In [21]:
engine = SubQuestionQueryEngine(timeout=200,verbose=True)
llm = OpenAI(model="gpt-4o")
result = await engine.run(
    llm=llm,
    tools = query_engine_tools,
    query = "How has the total amount of net profit for all 3 years?"
)

# print(result)

Running step query
Query is How has the total amount of net profit for all 3 years?
Step query produced no event
Running step sub_question
Sub-question is What is the net profit for the first year?
> Running step 719f0305-6fb3-40eb-8526-f76e6cdfdddc. Step input: What is the net profit for the first year?
Thought: The user is asking for the net profit for the first year. I need to use a tool to find this information.
Action: budget_2022
Action Input: {'input': 'net profit'}
Observation: The decrease in net profit for fiscal 2022 compared to fiscal 2021 was primarily due to a decrease in operating profit by 1.5% and a decrease in other income by 0.3% as a percentage of revenue, partially offset by a decrease of 0.7% in tax expense as a percentage of revenue.
> Running step 8678a737-a8ba-46da-a9bd-fac7df141a02. Step input: None
Thought: The provided information explains the reasons for the decrease in net profit for fiscal 2022 but does not provide the actual net profit value. I need to u

In [22]:
print(result)

The total amount of net profit for all three years is the sum of the net profits for each individual year. 

For the first year (fiscal 2022), the net profit is $2,637 million. 
For the second year (2023), the net profit is $2,983 million. 
For the third year (2024), the net profit is $3,169 million.

Adding these amounts together:

$2,637 million + $2,983 million + $3,169 million = $8,789 million.

Therefore, the total amount of net profit for all three years is $8,789 million.
